In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
#allows to print the dataframe nicely
from IPython.core import display as ICD
!pip install plotly
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls
from scipy import stats

In [ ]:
# import additional packages to insepct data and clean them
import pandas as pd
import os 
import random 
from zipfile import ZipFile
import datetime

In [ ]:
# import helping functions from the implementation file
#from proj1_helpers import load_csv_data
from proj1_helpers import *
import implementations
from additional_implementations import *
import seaborn as sns

In [ ]:
# import zipped files from the github repository
data_folder='./data/'
zip_file = ZipFile(data_folder+'all.zip')
# zip file creates a list of files with certain properties
zip_file.infolist()

In [ ]:
# now we want to access the 'filename' property in the zipfile variable
# and we create a dictionary of dataframe
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}
df_train=dfs['train.csv']
df_test=dfs['test.csv']
df_sample_submission=dfs['sample-submission.csv']

In [ ]:
df_train_pca=df_train.drop(columns=['Prediction', 'Id'])

In [ ]:
counts={}
for column_name in df_train_pca.columns.values:
    
    check=df_train_pca[column_name]==-999.0
    counts[column_name]=np.sum(check.astype('int'))
counts

In [ ]:
df_train_pca=df_train_pca.drop(columns=['PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'DER_lep_eta_centrality', 'DER_prodeta_jet_jet', 'DER_mass_jet_jet', 'DER_deltaeta_jet_jet','PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi'])

In [ ]:
median=np.median(df_train_pca['DER_mass_MMC'])
df_train_pca['DER_mass_MMC'][df_train_pca['DER_mass_MMC']==-999.0]=median

### PCA

In [ ]:
features = df_train_pca.iloc[:,1:].columns.values
df_train_pca_std=pd.DataFrame()

for feature in features:
    
    df_train_pca_std[feature]=standardize_personal(df_train_pca[feature])
df_train_pca_std.head()

In [ ]:
cov_df_train_pca_std=np.cov(df_train_pca_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_df_train_pca_std)

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

In [ ]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = Bar(
        x=['PC %s' %i for i in range(1,20)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,20)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance by different principal components')

fig = Figure(data=data, layout=layout)
py.iplot(fig)

In [ ]:
eig_pairs[0][1].reshape(20,1)
number_features=20
#decide how many principal components i get
number_pa=15
#define matrix to be filled in
matrix_w=np.ones((number_features, number_pa))
for i in range(number_pa):
    matrix_w[:,i] = eig_pairs[i][1]

In [ ]:
df_features_train_independent_std_transf = df_features_train_independent_std.dot(matrix_w)

In [ ]:
y_train_pca=np.array(df_train['Prediction'])
y_train_pca[np.where(y_train_pca=='b')] = -1.
y_train_pca[np.where(y_train_pca=='s')] = 1.
y_train_pca=y_train_pca.astype('float')

In [ ]:
tx_train_pca_polynomial=polynomial_features_simple(df_train_pca, 9)